<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/525_IRMOv2_riskAnalysis_utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Risk Analysis Utilities – Turning Risk Into a Programmable, Explainable Model

These utilities define how the orchestrator evaluates risk across the AI ecosystem.

Rather than treating risk as a vague label or subjective judgment, this module converts risk into a **transparent, weighted scoring model** that leadership can understand, audit, and adjust.

This is not probabilistic risk estimation.
It is **policy-driven risk evaluation**.

---

## Why This Design Matters

Most AI agents treat risk as:

* A label (“high”, “medium”, “low”)
* A heuristic
* Or an opaque model output

That makes them hard to trust and impossible to govern.

This agent does the opposite:

* Every component of risk is explicit
* Every score is traceable
* Every trade-off is visible

Executives can ask *why* a risk is critical — and get a clear answer.

---

## Normalization: Making Qualitative Signals Comparable

### Severity & Criticality Normalization

Human-readable labels like “high” or “medium” are useful, but they are not computable.

These utilities convert:

* **Risk severity**
* **Agent criticality**

into consistent 0–100 numeric scores.

This allows:

* Fair comparisons across risks
* Stable scoring logic
* Clear weighting without ambiguity

If leadership disagrees with how “high” maps to impact, they can change the mapping — not the logic.

---

## Time-Based Urgency: Risk That Ages Gets Louder

### Urgency Calculation

Risk is not static.

A medium issue ignored for 30 days can be more dangerous than a new critical one.

The urgency calculation:

* Increases risk weight as time passes
* Prevents “permanent medium” issues from being ignored
* Reflects operational reality

This ensures the agent escalates based on **inaction**, not just severity.

That’s a key executive concern.

---

## Impact Modeling: Not All Risks Are Equal

### Risk Type → Impact Mapping

Different risks hurt in different ways.

This model explicitly encodes that reality:

* Integration risks have the highest impact
* Security and compliance are weighted heavily
* Cost and operational risks still matter, but differently

Nothing is hidden.

These are **business judgments**, encoded directly into the system.

---

## Weighted Risk Scoring: Transparent Trade-Offs

### The Core Risk Equation

Each risk score is computed using four visible factors:

* Severity
* Agent criticality
* Business impact
* Time-based urgency

Each factor has an explicit weight.

This means:

* Risk prioritization is explainable
* Trade-offs are intentional
* Scores are consistent across runs

There is no hidden intelligence deciding what matters.

Leadership already decided — the agent simply applies it.

---

## Agent-Level Risk Assessment: From Signals to Decisions

### Aggregation & Classification

For each agent, risks are:

* Grouped by type
* Scored individually
* Aggregated into a total risk score

That score is then classified into:

* Medium
* High
* Critical

The thresholds are stable and predictable.

No surprises. No drifting behavior.

---

## Priority Actions: Focusing Attention Where It Matters

Instead of overwhelming users with every issue, the agent surfaces:

* The top 3 highest-scoring risks per agent
* With scores and context attached

This ensures:

* Executive attention is focused
* Teams know where to act first
* Risk discussions are grounded in evidence

This is risk management, not risk reporting.

---

## Portfolio-Wide Consistency

The `analyze_all_agent_risks` function applies the same logic uniformly across all agents.

There are:

* No special cases
* No dynamic heuristics
* No agent-specific hacks

This makes portfolio-level comparisons fair and defensible.

---

## Why Executives Care About This Approach

This risk model ensures the agent is:

* **Reliable** — same inputs, same outputs
* **Predictable** — thresholds and weights define behavior
* **Explainable** — every score can be decomposed
* **Programmable** — leadership intent is encoded
* **Safe to scale** — no black-box escalation

This is not an agent that *reacts*.
It is an agent that **applies policy consistently**.

---

## Architectural Takeaway

This module demonstrates a core principle of your system:

> **Risk is not something AI should “detect.”
> It is something organizations should define — and AI should enforce.**

By making risk explicit, weighted, and auditable, this orchestrator earns the right to be trusted.



In [ ]:
"""Risk analysis utilities"""

from typing import Dict, List, Any, Optional
from datetime import datetime


def normalize_severity(severity: str) -> float:
    """Convert severity string to numeric score (0-100)"""
    severity_map = {
        "critical": 100.0,
        "high": 75.0,
        "medium": 50.0,
        "low": 25.0
    }
    return severity_map.get(severity.lower(), 50.0)


def normalize_criticality(criticality: str) -> float:
    """Convert criticality string to numeric score (0-100)"""
    criticality_map = {
        "critical": 100.0,
        "high": 75.0,
        "medium": 50.0,
        "low": 25.0
    }
    return criticality_map.get(criticality.lower(), 50.0)


def calculate_risk_urgency(risk: Dict[str, Any]) -> float:
    """Calculate time-based urgency score (0-100)"""
    detected_at = risk.get("detected_at")
    if not detected_at:
        return 50.0  # Default urgency

    try:
        detected_time = datetime.fromisoformat(detected_at.replace("Z", "+00:00"))
        now = datetime.now(detected_time.tzinfo)
        days_open = (now - detected_time).days

        # Higher urgency for older risks
        if days_open >= 30:
            return 100.0  # Critical urgency
        elif days_open >= 14:
            return 75.0  # High urgency
        elif days_open >= 7:
            return 50.0  # Medium urgency
        else:
            return 25.0  # Low urgency
    except (ValueError, AttributeError):
        return 50.0


def calculate_risk_score(
    risk: Dict[str, Any],
    agent: Dict[str, Any],
    weights: Dict[str, float]
) -> float:
    """Calculate weighted risk score for a single risk"""
    severity_score = normalize_severity(risk.get("severity", "medium"))
    criticality_score = normalize_criticality(agent.get("criticality", "medium"))
    urgency_score = calculate_risk_urgency(risk)

    # Impact score based on risk type
    risk_type = risk.get("risk_type", "unknown")
    impact_map = {
        "integration": 100.0,  # Integration risks are high impact
        "operational": 75.0,
        "cost": 50.0,
        "compliance": 90.0,
        "security": 95.0
    }
    impact_score = impact_map.get(risk_type.lower(), 50.0)

    # Weighted score
    total_score = (
        severity_score * weights.get("severity", 0.40) +
        criticality_score * weights.get("criticality", 0.30) +
        impact_score * weights.get("impact", 0.20) +
        urgency_score * weights.get("urgency", 0.10)
    )

    return round(total_score, 1)


def assess_agent_risks(
    agent_id: str,
    risks: List[Dict[str, Any]],
    agent: Dict[str, Any],
    weights: Dict[str, float]
) -> Dict[str, Any]:
    """Assess all risks for a single agent"""
    # Group risks by type
    integration_risks = [r for r in risks if r.get("risk_type") == "integration"]
    operational_risks = [r for r in risks if r.get("risk_type") == "operational"]
    cost_risks = [r for r in risks if r.get("risk_type") == "cost"]
    other_risks = [r for r in risks if r.get("risk_type") not in ["integration", "operational", "cost"]]

    # Calculate scores for each risk
    all_risk_scores = []
    for risk in risks:
        score = calculate_risk_score(risk, agent, weights)
        all_risk_scores.append(score)

    # Total risk score (average of all risk scores, weighted by severity)
    if all_risk_scores:
        total_risk_score = sum(all_risk_scores) / len(all_risk_scores)
    else:
        total_risk_score = 0.0

    # Determine risk level
    if total_risk_score >= 75.0:
        risk_level = "critical"
    elif total_risk_score >= 50.0:
        risk_level = "high"
    else:
        risk_level = "medium"

    # Priority actions (risks that need attention)
    priority_actions = []
    for risk in sorted(risks, key=lambda r: calculate_risk_score(r, agent, weights), reverse=True)[:3]:
        priority_actions.append({
            "risk_id": risk.get("risk_id"),
            "risk_type": risk.get("risk_type"),
            "severity": risk.get("severity"),
            "signal": risk.get("signal"),
            "score": calculate_risk_score(risk, agent, weights)
        })

    return {
        "agent_id": agent_id,
        "integration_risks": integration_risks,
        "operational_risks": operational_risks,
        "cost_risks": cost_risks,
        "other_risks": other_risks,
        "total_risk_score": round(total_risk_score, 1),
        "risk_level": risk_level,
        "priority_actions": priority_actions
    }


def analyze_all_agent_risks(
    agents: List[Dict[str, Any]],
    risks_lookup: Dict[str, List[Dict[str, Any]]],
    weights: Dict[str, float]
) -> List[Dict[str, Any]]:
    """Analyze risks for all agents"""
    assessments = []
    for agent in agents:
        agent_id = agent["agent_id"]
        risks = risks_lookup.get(agent_id, [])
        assessment = assess_agent_risks(agent_id, risks, agent, weights)
        assessments.append(assessment)
    return assessments


In [ ]:
def risk_analysis_node(
    state: IntegrationRiskManagementOrchestratorState,
    config: IntegrationRiskManagementOrchestratorConfig
) -> Dict[str, Any]:
    """Risk Analysis Node: Assess risks for all agents"""
    errors = state.get("errors", [])
    agents = state.get("agents", [])
    risks_lookup = state.get("risks_lookup", {})

    if not agents:
        return {
            "errors": errors + ["risk_analysis_node: agents required"]
        }

    try:
        risk_assessments = analyze_all_agent_risks(
            agents,
            risks_lookup,
            config.risk_scoring_weights
        )

        return {
            "risk_assessments": risk_assessments,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"risk_analysis_node: {str(e)}"]
        }